#### Codificación acciones_ampliadas:
 + ((BAJAR, i, j),[a,b]) &rArr; El ascensor i baja j plantas desde el instante a y tarda b-a segundos
 + ((SUBIR, i, j),[a,b]) &rArr; El ascensor i sube j plantas desde el instante a y tarda b-a segundos
 + ((COGER_PERSONA, i, j=[]),[a]) &rArr; El ascensor i coge a las personas de la lista j en el instante a
 + ((DEJAR_PERSONA, i, j=[]),[a]) &rArr; El ascensor i deja a las personas de la lista j en el instante a
 + ((ESPERAR, i),[a,b]) &rArr; El ascensor i espera desde el instante a y tarda b-a segundos

#### CÓDIGOS OPERACIONES

In [54]:
BAJAR=0
SUBIR=1
COGER_PERSONA=2
DEJAR_PERSONA=3
ESPERAR=4 #Solo internamente en decodificador

***decodificador_acciones_paralelas***: Hace legible la lista de acciones

In [55]:
def decodificador_acciones_paralelas(acciones_ampliadas):
    lista_imprimir = []
    
    #Pasar de tener instantes a tener rangos [inicio,fin], comparando con la anterior (se presupone ordenado)
    #Añade la accion de ESPERA
    prev_instante = 0
    for i in range(len(acciones_ampliadas)):
        acc_ampl = acciones_ampliadas[i]
        accion = acc_ampl[0]
        instante = acc_ampl[1]
        if i > 0 and accion[1] != acciones_ampliadas[i - 1][0][1]:
            prev_instante = 0
        if accion[0] == COGER_PERSONA and instante > prev_instante:
            lista_imprimir.append(((ESPERAR, accion[1]), [prev_instante, instante]))
        if accion[0] == COGER_PERSONA or accion[0] == DEJAR_PERSONA:
            lista_imprimir.append((accion, [instante]))
        else:
            lista_imprimir.append((accion, [prev_instante, instante]))
        prev_instante = instante
        
    #Imprimir
    for accion_ampl_tiempo in lista_imprimir:
        tiempo = "[" + ','.join([str(t) for t in accion_ampl_tiempo[1]]) + "] -> "
        accion = accion_ampl_tiempo[0]
        if accion[0]==ESPERAR:
            accion_impresa = "El ascensor "+str(accion[1])+" espera"
        else:
            #Decodificador_acciones pero sin hacer print
            if accion[0]==BAJAR or accion[0]==SUBIR:
                if accion[0]==BAJAR:
                    sube_baja = " baja "
                else:
                    sube_baja = " sube "
                if accion[2]>1:
                    plantas = " plantas"
                else:
                    plantas = " planta"
                accion_impresa = "El ascensor "+str(accion[1])+sube_baja+str(accion[2])+plantas
            else:
                if accion[0]==COGER_PERSONA:
                    coge_deja = " coge a "
                else:
                    coge_deja = " deja a "
                lista_gente = [str(num) for num in accion[2]]
                if len(lista_gente)>1:
                    gente = "las personas "
                else:
                    gente = "la persona "
                accion_impresa = "El ascensor "+str(accion[1])+coge_deja+gente+", ".join(lista_gente)
        print(tiempo+accion_impresa)
    

***reordenar_acciones_paralelas***: Ordena las acciones poniéndolas agrupadas por ascensores

In [56]:
def reordenar_acciones_paralelas(acciones_ampliadas):
    clasificar_por_ascensor = [[] for _ in range(NUM_ASCENSORES)]
    for acc_ampl in acciones_ampliadas: #tuplas (accion,instante)
        clasificar_por_ascensor[acc_ampl[0][1]].append(acc_ampl)
    return [acc_ampl for lista_ascensor in clasificar_por_ascensor for acc_ampl in lista_ascensor] #[[]] a []

***quita_dejar_coger_inutiles_acciones_paralelas***: Quita si hay una cadena dejar-coger o coger-dejar para un mismo ascensor y persona sin que ese ascensor suba o baje en el medio

In [57]:
def quita_dejar_coger_inutiles_acciones_paralelas(acciones_ampliadas):
    index_ultimo_dejar_por_ascensor = [[-1 for _ in range(NUM_PERSONAS)] for _ in range(NUM_ASCENSORES)]
    indexes_a_borrar = set()
    for i, accion_ampliada in enumerate(acciones_ampliadas):
        accion = accion_ampliada[0]
        if accion[0]==DEJAR_PERSONA or accion[0]==COGER_PERSONA:
            if index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]==-1:
                index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]=i
            else:
                supuestamente_inverso = acciones_ampliadas[index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]][0]
                if supuestamente_inverso[0] != accion[0]:
                    indexes_a_borrar.add(index_ultimo_dejar_por_ascensor[accion[1]][accion[2]])
                    indexes_a_borrar.add(i)
                    index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]=-1
                else: #IMPOSIBLE, cogeríamos (respect. dejaríamos) a la misma persona en el mismo ascensor dos veces
                    index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]=i
                    
        else:
            index_ultimo_dejar_por_ascensor[accion[1]] = [-1 for _ in range(NUM_PERSONAS)] #Ya no cancelables
    
    nuevas_acciones = []
    for i in range(len(acciones)):
        if not(i in indexes_a_borrar): #asi mejor para que no se corran los indices y porque del/remove/pop tienen coste lineal
            nuevas_acciones.append(acciones_ampliadas[i])
    
    return nuevas_acciones

***fusionador_acciones_paralelas***: Junta dos acciones iguales seguidas:
 + Si un mismo ascensor coge (o deja persona) a m y n seguidamente: aparecerá que se cogió a las personas m,n
 + Si un mismo ascensor sube (o baja) a m y a n seguidamente: aparecerá que subió m+n plantas

In [58]:
def fusionador_acciones_paralelas(acciones_ampliadas):
    nuevas_acciones = []
    for acc_ampl in acciones_ampliadas:
        accion = acc_ampl[0]
        instante = acc_ampl[1]
        if accion[0]==COGER_PERSONA or accion[0]==DEJAR_PERSONA:
            accion = (accion[0],accion[1],[accion[2]])
            
        if bool(nuevas_acciones) and (accion[0] == nuevas_acciones[-1][0][0] and accion[1] == nuevas_acciones[-1][0][1]):
            if accion[0]==SUBIR or accion[0]==BAJAR:
                nuevo_instante = nuevas_acciones[-1][1]+instante
            else:
                nuevo_instante = instante
            # Si es SUBIR o BAJAR suma las plantas, si es COGER o DEJAR concatena las listas de personas
            nuevas_acciones[-1] = ((nuevas_acciones[-1][0][0], nuevas_acciones[-1][0][1], nuevas_acciones[-1][0][2] + accion[2]), nuevo_instante)
        else:
            nuevas_acciones.append((accion,instante))
    return nuevas_acciones

***cocina_acciones_paralelas***: Reduce la lista de acciones lo máximo posible

In [59]:
def cocina_acciones_paralelas(acciones_ampliadas):
    acciones_amp1 = reordenar_acciones_paralelas(acciones_ampliadas)
    acciones_amp2 = quita_dejar_coger_inutiles_acciones_paralelas(acciones_amp1)
    acciones_amp3 = fusionador_acciones_paralelas(acciones_amp2)
    return acciones_amp3

***coste_acciones_paralelas***: Calcula el coste paralelo, esto es, el máximo entre los tiempos de finalización de cada ascensor

In [60]:
def coste_acciones_paralelas(init,acciones):
    state = init
    for acc in acciones:
        state = problem.result(state,acc)
    return max(state.ascensores_tiempo_llegada)

***ampliadas_acciones_paralelas***: Dada la lista de acciones de la solución, devuelve una lista de las mismas acciones con el instante en el que se ejecuta esa acción en la solución 

In [61]:
def ampliadas_acciones_paralelas(init,acciones):
    state = init
    acciones_ampliadas = []
    for acc in acciones:
        state = problem.result(state,acc)
        acciones_ampliadas.append((acc,state.ascensores_tiempo_llegada[acc[1]]))
    return acciones_ampliadas